# Species distribution modeling with multimodal satellite and environmental data

Welcome to this wonderfull main to discover the beauty of mother earth, enjoy your work and be happy :)

## 1. Setup

## 2. Data loading

In [ ]:
from Aurelien_DataLoader.GeoPlantDataset import GeoPlantDataset, viz_sample
train=GeoPlantDataset(data_folder=f"{root}/data", split='train')

## 3. Create model

In [ ]:
# neural network with 3 NN (for each modality) and a classifier at the end

class multimodal_SDM (nn.Module): #heritates from class nn.Module
    
    def __init__(self, dim_NN_env=128, dim_NN_sat=4096, dim_NN_timeseries=64): #dimension of output of neural networks
        super(multimodal_SDM, self).__init__()  #call the init of the parent class
        
        self.dim_NN_env=dim_NN_env

        self.dim_NN_sat=dim_NN_sat

        self.dim_NN_timeseries=dim_NN_timeseries
        
        self.MLP_env = nn.Sequential( #19 bioclim variables to 128 values
            nn.Linear(19,50),
            nn.ReLU(),
            nn.Linear(50, 50),
            nn.ReLU(),
            nn.Linear(50, 50),
            nn.ReLU(),
            nn.Linear(50, 128),
            nn.ReLu()
            )
        
        self.CNN_sat= #direct resnet something

        self.CNNtime_series= nn.Sequential( 
            ''' 
            R G B NIR with 10 years and 4 seasons= 40 values: 4 channels, length 40
            Like Alexnet but shorter (2 convolutionnal layers) and in 1D
            ''' 
            nn.Conv1d(in_channels=4, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(32*10, dim_NN_timeseries), # 32 channels * length 10 (2 poolings of size 2 equivalent to divide /4 the length of the input) ?
            nn.ReLU()
        )

        self.classifier= nn.Sequential(
            nn.Linear(dim_NN_env+dim_NN_sat+dim_NN_timeseries, 1000)
            nn.ReLU(),
            nn.Linear(1000, 342)
            nn.Sigmoid()
        )

    def forward (self,x):
        '''x is a GeoPlantDataset object containing the 3 modalities:
        - env_variables
        - satellite_patches
        - landsat_timeseries
        '''
        env_variables=train.env_variables
        satellite_patches=train.satellite_patches
        landsat_timeseries=train.landsat_timeseries

        #pass each modality through its NN
        NN_env_out=self.MLP_env(env_variables)
        NN_sat_out=self.CNN_sat(satellite_patches)
        NN_time_series_out=self.CNN_timeseries(landsat_timeseries)

        #concatenate the outputs
        combined=torch.cat((NN_env_out, NN_sat_out, NN_time_series_out), dim=1)

        #pass through the classifier
        output=self.classifier(combined)
        return output

## 4. Evaluation of the model